In [1]:
# To leverage sample dataset stored in Azure Cosmos DB for MongoDB vCore to ground OpenAI models by using Mongo DB vector similarity search functionality
# Create a chat session with GPT 3.5 completions model to answer questions about Azure services informed by dataset


In [2]:
import json
import datetime
import time

from azure.core.exceptions import AzureError
from azure.core.credentials import AzureKeyCredential
import pymongo

import openai
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [ ]:
from dotenv import dotenv_values

# specify the name of the .env file name
env_name = "llm.env" # following example.env template change to our own .env file name
config = dotenv_values(env_name)

cosmosdb_endpoint = config["cosmos_db_api_endpoint"]
cosmosdb_key = config["cosmos_db_api_key"]
cosmosdb_connection_str = config['cosmos_db_connection_string']

COSMOS_MONGO_USER = config['cosmos_db_mongo_user']
COSMOS_MONGO_PWD = config['cosmos_db_mongo_pwd']
COSMOS_MONGO_SERVER = config['cosmos_db_mongo_server']

openai.api_type = config['openai_api_type']
openai.api_key = config['openai_api_key']
openai.api_base = config['openai_api_endpoint']
openai.api_version = config['openai_api_version']
embeddings_deployment = config['openai_embeddings_deployment']
completions_deployment = config['openai_completions_deployment']


In [ ]:
# Creat an Azure Cosmos DB for MongoDB vCore resource
# Load data and create embeddings

# Load text-sample.json data file
data_file = open(file="../../Dataset/AzureServices/text-sample.json", mode="r")
#data_file = open(file="../../DataSet/AzureServices/text-sample_w_embeddings.json", mode="r") # load this file instead if embeddings were previously created and saved.
data = json.load(data_file)
data_file.close()

In [ ]:
# Take a peek at one data item
print(data[0])


In [ ]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(10))
def generate_embeddings(text):
    '''
    Generate embeddings from string of text
    This will be used to vectorize data and user input for interactions with Azure OpenAI.
    '''
    response = openai.Embedding.create(
        input=text, engine="text-embedding-ada-002"
    embeddings = response['data'][0]['embedding']
    time.sleep(0.5) # rest period to avoid rate limiting on AOAI for free tier
    return embeddings
    )

In [ ]:
# Genearte embeddings for title and content fields
n = 0
for item in data:
    n+=1
    title = item['title']
    content = item['content']
    title_embeddings = generate_embeddings(title)
    content_embeddings = generate_embeddings(content)
    item['titleVector'] = title_embeddings
    item['contentVector'] = content_embeddings
    item['@search.action'] = 'upload'
    print("Creating embeddings for item:", n, "/", len(data), end='\r')

# Save embeddings to sample_text_w_embeddings.json file
with open("../../DataSet/AzureServices/text-sample_w_embeddings.json", "w") as f:
    json.dump(data, f)

In [ ]:
#Connect and set up Cosmos DB for MongoDB vCore 
# Set up the connection

mongo_conn = "mongodb+srv://"+COSMOS_MONGO_USER+":"+COSMOS_MONGO_PWD+"@"+COSMOS_MONGO_SERVER+"?tls=true&authMechanism=SCRAM-SHA-256&retrywrites=false&maxIdleTimeMS=120000"
mongo_client = pymongo.MongoClient(mongo_conn)


In [ ]:
# Set up the DB and collection 

# create a database called TutorialDB
db = mongo_client['TutorialDB']

# Create collection if it doesn't exist
COLLECTION_NAME = "TutorialCol"

collection = db[COLLECTION_NAME]

if COLLECTION_NAME not in db.list_collection_names():
    # Creates a unsharded collection that uses the DBs shared throughput
    db.create_collection(COLLECTION_NAME)
    print("Created collection '{}'.\n".format(COLLECTION_NAME))
else:
    print("Using collection: '{}'.\n".format(COLLECTION_NAME))

In [ ]:
# Create the vector index 
db.command({
  'createIndexes': 'TutorialCol',
  'indexes': [
    {
      'name': 'vectorSearchIndex',
      'key': {
        "contentVector": "cosmosSearch"
      },
      'cosmosSearchOptions': {
        'kind': 'vector-ivf',
        'numLists': 1,
        'similarity': 'COS',
        'dimensions': 1536
      }
    }
  ]
});

In [ ]:
# Update data to the collection
# A simple insert_many() to insert our data in JSON format into the newly created DB and collection
collection.insert_many(data)


In [ ]:
def ask_question(query, nr_of_results=1):
    query = generate_embeddings(query)
    embeddings_list = []
    pipeline = [
        {
            '$search': {
                "cosmosSearch": {
                    "vector": question_embeddings,
                    "path": "contentVector",
                    "k": nr_of_results
                },
                "returnStoredSource": True
            }
        }
    ]
    results = collection.aggregate(pipeline)
    for item in results:
        print(item['title'])
        print(item['content'])
        print("\n")

In [ ]:
# Vector Search in Cosmos DB for MongoDB vCore
# Simple function to assist with vector search
def vector_search(query, num_results=1):
    query_embedding = generate_embeddings(query)
    embeddings_list = []
    pipeline = [
        {
            '$search': {
                "cosmosSearch": {
                    "vector": query_embedding,
                    "path": "contentVector",
                    "k": num_results
                },
                "returnStoredSource": True
            }
        }
    ]
    results = collection.aggregate(pipeline)
    return results

In [ ]:
query = "tools for software development"  
results = vector_search(query)
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  

In [ ]:
# Q and A over the data with GPT-3.5
# To feed prompt into the Completions model. Then we will create interactive loop where we can pose questions to the model and receive information grounded in our data
# This function helps
def generate_completion(prompt):
    system_prompt = '''
    You are an intelligent assistant for Microsoft Azure services.
    You are designed to provide helpful answers to user questions about Azure services given the information about to be provided.
        - Only answer questions related to the information provided below, provide 3 clear suggestions in a list format.
        - Write two lines of whitespace between each answer in the list.
        - Only provide answers that have products that are part of Microsoft Azure.
        - If you're unsure of an answer, you can say ""I don't know"" or ""I'm not sure"" and recommend users search themselves."
    '''

    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input},
    ]

    for item in results:
        messages.append({"role": "system", "content": prompt['content']})

    response = openai.ChatCompletion.create(engine=completions_deployment, messages=messages)
    
    return response
                                                                                                                                   

In [ ]:
# Create a loop of user input and model output. You can now perform Q&A over the sample data!

user_input = ""
print("*** Please ask your model questions about Azure services. Type 'end' to end the session.\n")
user_input = input("Prompt: ")
while user_input.lower() != "end":
    results_for_prompt = vector_search(user_input)
    completions_results = generate_completion(results_for_prompt)
    print("\n")
    print(completions_results['choices'][0]['message']['content'])
    user_input = input("Prompt: ")